# **Generation Similarity**

In [1]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma langchain_groq ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.2/163.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 40.2 MB/s eta

In [2]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_core.output_parsers import StrOutputParser
import re
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.storage import InMemoryStore
from operator import itemgetter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from pydantic import BaseModel, Field
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
from langchain import PromptTemplate
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from ragas import SingleTurnSample
from ragas.metrics import ResponseRelevancy, LLMContextRecall
from ragas.llms import LangchainLLMWrapper
import math
import numpy as np

In [3]:
uploaded_files = files.upload()

Saving model_results_docs_feed_15_parent_chunk_size_360.json to model_results_docs_feed_15_parent_chunk_size_360.json
Saving model_response_docs_feed_15_parent_chunk_size_360.json to model_response_docs_feed_15_parent_chunk_size_360.json
Saving generation_eval.json to generation_eval.json


In [4]:
model_response_name = 'model_response_docs_feed_15_parent_chunk_size_360.json'
model_results_name = 'model_results_docs_feed_15_parent_chunk_size_360.json'

In [5]:
with open(model_response_name, 'r') as file:
    responses = json.load(file)

In [6]:
with open(model_results_name, 'r') as file:
    results = json.load(file)

In [7]:
with open('generation_eval.json', 'r') as file:
    generation_eval = json.load(file)

cross_encoder_embedings_model_name = "BAAI/bge-reranker-large"
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)

def process_response_answer_only(response):
  answer_only = response.split('Citations')[0].replace("\n", " ").strip()
  return answer_only

question_numbers = ['question_1', 'question_2', 'question_3', 'question_4', 'question_5']
similarity_gpt_and_ours = []
for question_number in question_numbers:
  parent_question = generation_eval[question_number]['question']
  ground_truth_answer = generation_eval[question_number]['ground_truth']
  sub_answers = []
  for response in responses:
    for k,v in response.items():
      if response[k]['parent_question'] == parent_question:
        sub_answers.append(process_response_answer_only(response[k]['answer']))
  model_answer = '.'.join(sub_answers)

  similarity_gpt_and_ours.append(cross_encoder_model.score((ground_truth_answer,model_answer)))

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [8]:
similarity_gpt_and_ours

[0.050902642, 0.0064415657, 0.21006925, 0.60452485, 0.999718]

In [9]:
results['similarity_with_ground_truth'] = {
    'all_scores': [float(x) for x in similarity_gpt_and_ours],
    'mean': float(np.mean(similarity_gpt_and_ours))
}

In [10]:
results

{'mean_response_relevancy_score': 0.835963441727246,
 'overall_citation_recall_val': 0.4910726310726311,
 'overall_citation_precision_val': 0.971851851851852,
 'f1_cg': 0.6524599888404953,
 'recall_answer': 1.0,
 'precision_answer': 1.0,
 'f1_answer': 1.0,
 'recall_refusal': 0.8,
 'precision_refusal': 1.0,
 'f1_refusal': 0.888888888888889,
 'f1_rg': 0.9444444444444444,
 'similarity_with_ground_truth': {'all_scores': [0.05090264230966568,
   0.0064415656961500645,
   0.21006925404071808,
   0.6045248508453369,
   0.9997180104255676],
  'mean': 0.3743312954902649}}

In [11]:
with open(model_results_name, 'w') as file:
    json.dump(results, file, indent=4)